# PROYECTO 2023

Descripción proyecto

## Importar librerías

In [1]:
# General
import numpy as np
import math
import os

# Procesamiento de imagenes
import cv2

# Visualizacion
from   tqdm.auto import tqdm
import matplotlib.pyplot as plt

from   balu3.fx.chr    import lbp
from   balu3.ft.norm   import minmax
from   balu3.fs.sel    import sfs,clean
from   balu3.io.misc   import imageload
from scipy.stats import mode
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Parámetros Extracción

In [2]:
# LBP
LBP_HDIV = LBP_VDIV = 8
LBP_BINS = 59

# Fetaures
features_per_function = {
    "lbp": LBP_HDIV * LBP_VDIV * LBP_BINS
}
print(features_per_function)

{'lbp': 3776}


### Parámetros Selección

### Funciones auxiliares

In [3]:
def extract_features(color_mode, dataset_type, feature_type):
    # Cargar rutas de imágenes y etiquetas según el dataset_type
    # ...
    K = 90   # <= NUMERO DE CLASES

    if dataset_type == 'train':
        fpath = "G00"     # <= DIRERCTORIO DE LA BASE DE DATOS
        N = 12   # <= NUMERO DE IMAGENES POR CLASE
        n = range(12)

    elif dataset_type == 'test':
        fpath = "G01"     # <= DIRERCTORIO DE LA BASE DE DATOS
        N = 4   # <= NUMERO DE IMAGENES POR CLASE
        n = range(12, 16)

    elif dataset_type == 'test2':
        fpath = "G02"     # <= DIRERCTORIO DE LA BASE DE DATOS
        N = 4   # <= NUMERO DE IMAGENES POR CLASE
        n = range(16, 20)

    dig_clase = 3     # <= NÚMERO DE DÍGITOS POR CLASE
    dig_img = 3     # <= NÚMERO DE DÍGITOS POR NÚMERO DE IMAGEN
    prefix = "ID"     # <= PREFIJO DEL NOMBRE DEL ARCHIVO DE LA IMAGEN
    imprefix = fpath + '/' + prefix

    # ground truth (clasificacion ideal)
    y = np.zeros((K*N), 'int')
    features = np.zeros((K*N, features_per_function[feature_type]))

    t = 0
    for j in tqdm(range(K)):                  # para cada clase
        for i in n:                # para cada imagen de la clase
            # Lectura de la imagen
            clase = j+1
            num_img = i+1

            img = imageload(imprefix, clase, dig_clase, num_img, dig_img)
            size = img.shape[0]
            new_size = int(size // 1.2)
            img = img[size-new_size: new_size,
                      size-new_size: new_size]
            
            y[t] = j+1

            # Preprocesar las imágenes según el color_mode (blue, red, green, blue)
            if color_mode == 'gray':
                img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            elif color_mode == 'red':
                img = img[:, :, 0]
            else:
                raise ValueError(f"Invalid color mode: {color_mode}")

            # Extraer características según el feature_type
            if feature_type == 'lbp':
                # Aplicar LBP a cada imagen
                features[t,:] = lbp(img, hdiv=LBP_HDIV, vdiv=LBP_VDIV)

            else:
                raise ValueError(f"Invalid feature_type: {feature_type}")


            t = t+1
    # Devolver las características extraídas y las etiquetas
    return features, y


def load_classifier(classifier):
    if classifier == 'lda':
        return LinearDiscriminantAnalysis()
    else:
        raise ValueError(f"Invalid classifier: {classifier}")


## Extracción y selección de caracteristicas

In [4]:
def extraction_main(extraction_models, color_modes, image_set):
    # Create the "features" folder if it doesn't exist
    if not os.path.exists("features"):
        os.makedirs("features")
    if not os.path.exists("y"):
        os.makedirs("y")

    # Iterate over extraction models and color modes
    for model in tqdm(extraction_models):
        for color_mode in color_modes:
            # Extract features
            (X, y) = extract_features(color_mode, image_set, model)

            # Save features to files
            filename = f"features/X_{image_set}_{model}_{color_mode}.npy"
            y_filename = f"y/y_{image_set}.npy"

            np.save(filename, X)
            np.save(y_filename, y)

## Clasificación

### De acá en adelante se pueden ejecutar el modelo

## MODELO 2

descripción del modelo

In [5]:
def model_2(X_train, y_train):
    models = [load_classifier('lda')]
    
    best_model = None
    best_accuracy = 0.0
    # Iterar sobre diferentes modelos y encontrar el que maximice la precisión en cross-val
    for model in models:
        # Realizar cross-val con 4 folds en las imágenes de grupo 0
        scores = cross_val_score(model, X_train, y_train, cv=4)

        # Calcular la precisión promedio en cross-val
        accuracy = scores.mean()

        # Actualizar el mejor modelo si se encuentra uno con una precisión mayor
        model.fit(X_train, y_train)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

    # Entregar el mejor modelo encontrado
    entregable_modelo_2 = best_model

    return models, entregable_modelo_2

## Código para ejecutar tests

In [32]:
EXTRACTION_MODS = ['lbp']
COLOR_MODES = ["gray", "red"]

def test_set(test_set):

    # MODO DE EJECUCION
    
    extraction_main(EXTRACTION_MODS, COLOR_MODES, test_set) #descomentar para train y test

    y_train = np.load('y/y_train.npy')
    y_test = np.load(f'y/y_{test_set}.npy')

    X_train_final = []
    X_test_final = []
    
    for color in COLOR_MODES:

        X_train = []
        X_test = []
        
        for ext in EXTRACTION_MODS:
            X_train.append( np.load(f'features/X_train_{ext}_{color}.npy') )
            X_test.append( np.load(f'features/X_{test_set}_{ext}_{color}.npy') )

        X_train = np.concatenate(X_train, axis=1)
        X_test = np.concatenate(X_test, axis=1)

        sclean = clean(X_train)
        X_train_clean = X_train[:,sclean]
        X_test_clean = X_test[:,sclean]

        X_train_norm, a, b = minmax(X_train_clean)
        X_test_norm = X_test_clean * a + b

        X_train_final.append(X_train)
        X_test_final.append(X_test)

    X_train = np.concatenate(X_train_final, axis=1)
    X_test = np.concatenate(X_test_final, axis=1)

    ## MODELO 2
    models, best_model_2 = model_2(X_train, y_train)
    y_pred_2 = best_model_2.predict(X_test)
    accuracy_m2 = accuracy_score(y_test, y_pred_2)

    print(f"Accuracy model 2: {accuracy_m2}")

##### (OPCIONAL) G00

In [ ]:
!wget https://www.dropbox.com/s/s4opefjionbdbab/G00.zip
!unzip -qq G00.zip

##### (OPCIONAL) G01

In [ ]:
!wget https://www.dropbox.com/s/zur2wxzcce4qlgf/G01.zip
!unzip -qq G01.zip

##### ARCHIVOS NECESARIOS CON CARACTERISTICAS DE TRAIN

In [ ]:
!wget https://www.dropbox.com/s/3skad8oxal12a8l/features.zip
!wget https://www.dropbox.com/s/tidpv7sdluios86/y.zip

!unzip -qq features.zip
!unzip -qq y.zip

##### INSERTAR ACÁ EL LINK PARA G02, O SIMPLEMENTE PONER LA CARPETA G02 EN EL MISMO DIRECTIORIO QUE EL NOTEBOOK

In [ ]:
!wget <url>
!unzip -qq G02.zip

##### Consideraciones/Explicaciones: las caracteristicas de train vienes extraidas y están en la carpeta "features", entonces al ejecutar "test_set"  con "test2" se ejecuta la extraccion de las imagenes en la carpeta G02, y luego se cargan Train y Test recién extraido, se limpian y normalizan, se concatenan respectivamente y se ejecuta el modelo de clasificación.

In [30]:
TEST = "test2"
test_set(TEST)

Accuracy model 2: 0.85
